In [2]:
!pip3.10 install -r requirements.txt
# !pip3.10 install transformers -U

# !ollama pull llava-phi3
# !ollama pull llama3.2-vision 
# !ollama pull moondream
# !ollama pull llama3.2:1b
# !ollama pull llama3.2
# !ollama pull granite3.1-moe:3b
# !ollama pull smollm2:1.7b
# !ollama pull smollm2:1.7b-instruct-fp16

Defaulting to user installation because normal site-packages is not writeable


In [ ]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

/Users/lekshmanannatarajan/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
sequence_to_classify = "one day I will see the world"
candidate_labels = ['travel', 'cooking', 'dancing']
classifier(sequence_to_classify, candidate_labels)